# Project 5: chatbot con conocimiento privado

## Executive Summary

Crea un trabajador del conocimiento en tus datos para impulsar la productividad
- Reune los archivos necesarios en un solo lugar, la base de conociminetos personal
- vectoriza todo en chroma, el almacen de vectores propio
- crea un asistente de IA conversacional y formula preguntas

**Core Innovation:**

**Key Capabilities:**

**Business Value:**

**Technical Stack:**


In [1]:
# imports

import os, shutil
import glob
from dotenv import load_dotenv
import gradio as gr
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# imports for langchain, plotly and Chroma

from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from sklearn.manifold import TSNE
from pathlib import Path
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings

/opt/conda/envs/LLM/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# 1) Cargar PDFs (desde knowledge-base/)

KB_DIR = Path("knowledge-base").resolve()
print("KB_DIR:", KB_DIR)

loader = DirectoryLoader(
    str(KB_DIR),
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
)

documents = loader.load()
print("Docs (páginas) cargadas:", len(documents))
print("Ejemplo metadata:", documents[0].metadata if documents else None)

KB_DIR: /workspace/giova/llms-engineering-main/knowledge-base


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]

Docs (páginas) cargadas: 56
Ejemplo metadata: {'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'creator': 'Epic Editor v. 5.0', 'creationdate': '2011-12-14T14:40:39+00:00', 'author': 'SPSS Inc.', 'moddate': '2011-12-14T14:40:39+00:00', 'title': 'Manual CRISP-DM de IBM SPSS Modeler', 'source': '/workspace/giova/llms-engineering-main/knowledge-base/crisdm.pdf', 'total_pages': 56, 'page': 0, 'page_label': '1'}


In [6]:
# 2) Chunking

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print("Chunks:", len(chunks))
print("Snippet chunk:", chunks[0].page_content[:200] if chunks else None)

Chunks: 55
Snippet chunk: i
Manual CRISP-DM de IBM SPSS
Modeler


In [8]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 55 documents


In [9]:
collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 55 vectors with 1,536 dimensions in the vector store


In [12]:
# 5) Batería de preguntas (ejecución automática + fuentes)

test_questions = [
    "¿Qué significa CRISP-DM y para qué se usa?",
    "¿Cuántas fases tiene CRISP-DM?",
    "Enumera las 6 fases de CRISP-DM en orden.",
    "¿Qué se hace en la fase de Comprensión del negocio?",
    "¿Qué se hace en la fase de Comprensión de los datos?",
    "¿Qué tareas típicas se mencionan en Preparación de datos?",
    "¿Por qué CRISP-DM se considera un proceso iterativo?",
    "Dame un resumen de 'Comprensión del negocio' y cita páginas.",
    "Explica qué dice el documento sobre riesgos/contingencias y cita páginas.",
    "Ok, si mi objetivo es reducir abandono de clientes, ¿qué fase hago primero y por qué?",
    "Ahora reescribe ese plan en 6 bullets alineados a las fases CRISP-DM.",
]

## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [13]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']

# doc_type seguro (fallback)
doc_types = [m.get("doc_type", "unknown") for m in metadatas]

# Genera un color por cada tipo distinto (sin asumir nombres)
unique_types = sorted(set(doc_types))
palette = ["blue", "green", "red", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

type_to_color = {t: palette[i % len(palette)] for i, t in enumerate(unique_types)}
colors = [type_to_color[t] for t in doc_types]


In [14]:
pio.renderers.default = "iframe"  # el más robusto en JupyterLab remoto

In [15]:
# --- Asegura que vectors sea 2D numpy array ---
vectors = np.array(vectors)
assert vectors.ndim == 2, f"vectors debe ser 2D, llegó: {vectors.shape}"

n = vectors.shape[0]
if n < 2:
    raise ValueError(f"No hay suficientes embeddings para visualizar (n={n}).")

# --- t-SNE: ajusta perplexity según cantidad de puntos ---
# Regla: perplexity < n, típicamente <= (n-1)/3
perplexity = min(30, max(2, (n - 1) // 3))
tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity, init="pca", learning_rate="auto")
reduced_vectors = tsne.fit_transform(vectors)

# --- Hover text para PDF: fuente + página + tipo + snippet ---
def safe_snippet(text, k=200):
    if not text:
        return ""
    text = text.replace("\n", " ").strip()
    return (text[:k] + "...") if len(text) > k else text

hover_text = []
for md, t, d in zip(metadatas, doc_types, documents):
    src = os.path.basename(md.get("source", "unknown"))
    page = md.get("page", md.get("page_number", ""))
    page_str = f"{page}" if page != "" else "?"
    hover_text.append(
        f"Type: {t}"
        f"<br>Source: {src}"
        f"<br>Page: {page_str}"
        f"<br>Text: {safe_snippet(d)}"
    )

# --- Plot 2D ---
fig = go.Figure(
    data=[go.Scatter(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        mode="markers",
        marker=dict(size=6, color=colors, opacity=0.85),
        text=hover_text,
        hoverinfo="text"
    )]
)

fig.update_layout(
    title=f"2D Chroma Vector Store (t-SNE, n={n}, perplexity={perplexity})",
    xaxis_title="t-SNE x",
    yaxis_title="t-SNE y",
    width=900,
    height=650,
    margin=dict(r=20, b=20, l=20, t=60)
)

fig.show()


In [16]:
pio.renderers.default = "iframe"  # para JupyterLab remoto

vectors = np.array(vectors)
n = vectors.shape[0]
assert n == 55, f"Esperaba 55, llegó {n}"

tsne = TSNE(
    n_components=3,
    random_state=42,
    perplexity=15,      # recomendado para n=55
    init="pca",
    learning_rate="auto",
)
reduced_vectors = tsne.fit_transform(vectors)

fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='t-SNE x', yaxis_title='t-SNE y', zaxis_title='t-SNE z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Time to use LangChain to bring it all together

In [10]:
# create a new Chat with OpenAI
llm = ChatOpenAI(model=MODEL, temperature=0.7)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/tmp/ipykernel_3251/1344145089.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [11]:
# Let's try a simple question

query = "¿Qué se hace en la fase de Comprensión del negocio?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

En la fase de Comprensión del negocio, se realizan varias actividades clave para establecer un marco claro para el proyecto de minería de datos. Estas actividades incluyen:

1. **Definir objetivos comerciales**: Se identifican y documentan los objetivos específicos que la organización espera alcanzar con la minería de datos. Esto ayuda a alinear los esfuerzos del proyecto con las metas empresariales.

2. **Valorar la situación actual**: Se evalúa la situación comercial existente, incluyendo la disponibilidad de datos, recursos humanos, problemas existentes y factores de riesgo. Esto permite tener una visión clara de los recursos y limitaciones del proyecto.

3. **Identificar áreas problemáticas**: Se determina el área específica que necesita atención, como marketing, atención al cliente o desarrollo comercial, y se describe el problema general que se busca resolver.

4. **Compilación de información**: Se recopila información sobre la empresa, como la estructura organizativa, los recurs

In [17]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [18]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a32eeddc47c7c6bf9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
